# Hyperfield Additive Table Generator

This notebook contains a Python script to generate the additive table for quotient hyperfields arising as GF(p^k) / G_d, where G_d is the multiplicative subgroup of GF(p^k) of order d.

## Requirements

- Python 3.6+
- pandas (`pip install pandas`)
- galois (`pip install galois`)

## Usage

1.  **Install dependencies:** If you haven't already, run the cell with `pip install galois`. pandas is usually pre-installed in Colab.
2.  **Run the main function:** Execute the code cell containing the script. The `main()` function will prompt you to enter:
    *   A prime number `p`.
    *   An integer `k >= 1`.
    *   A divisor `d` of `p^k - 1`.
3.  **View the output:** The script will print the additive hyperoperation table for the specified quotient hyperfield as a pandas DataFrame. It will also print the characteristic and c-characteristic of the hyperfield.

## How it Works

The script utilizes the `galois` library to construct the finite field GF(p^k). It then finds a primitive element and constructs the multiplicative subgroup of order `d`. Coset representatives of GF(p^k)^*/G_d are determined, and the additive hyperoperation arising by Krasner's quotient construction is computed for all pairs of cosets (including the zero element).

## License

This project is licensed under the MIT License - see the [LICENSE](https://opensource.org/licenses/MIT) file for details.

In [ ]:
pip install galois

In [7]:
"""
Hyperfield additive table generator.

This module can generate the additive table for quotient hyperfields arising as
GF(p^k) / G_d, where G_d is the multiplicative subgroup of GF(p^k) of order d.

Requirements:
- pandas
- galois (pip install galois)

CLI usage:
- Run the module and input integers p, k, d such that p is prime, k >= 1, and d | (p^k - 1).
- The program constructs GF(p^k), builds the subgroup of order d, computes coset
  representatives of GF(p^k)^*/G_d, and prints the additive hyperoperation table including 0.

Notes and edge cases:
- Sorting and display: Field elements are cast to int for sorting when formatting cells.
  This reflects the vector-space representation chosen by galois (canonical integer mapping).
- For large fields or large d, the table can be very large; generation may be slow and memory intensive.
- If 'galois' is not installed, the generalized functionality will raise an ImportError.

Legacy (prime field) helpers for GF(p) modulo a subgroup of (Z/pZ)^* are retained below for reference.
"""

import pandas as pd
try:
    import galois
except ImportError as e:
    galois = None
try:
    from IPython.display import display as _ipython_display
    def display(obj):  # noqa: A001 - shadow built-in name intentionally for notebooks
        _ipython_display(obj)
except Exception:
    def display(obj):
        try:
            print(obj.to_string())
        except Exception:
            print(obj)

def is_prime(n):
    if n < 2:
        return False
    if n == 2:
        return True
    if n % 2 == 0:
        return False
    r = int(n**0.5)
    for i in range(3, r+1, 2):
        if n % i == 0:
            return False
    return True

def find_primitive_root(p):
    # Finds a primitive root modulo p (generator of the multiplicative group)
    # Works only for prime p
    if p == 2:
        return 1
    phi = p - 1
    factors = prime_factors(phi)
    for g in range(2, p):
        if all(pow(g, phi // f, p) != 1 for f in factors):
            return g
    raise ValueError("No primitive root found (should not happen for prime p)")

def prime_factors(n):
    # Returns a list of prime factors of n (without multiplicity)
    factors = []
    d = 2
    while d * d <= n:
        if n % d == 0:
            factors.append(d)
            while n % d == 0:
                n //= d
        d += 1 if d == 2 else 2
    if n > 1:
        factors.append(n)
    return factors

def construct_subgroup(p, d, g):
    # Constructs the subgroup G of order d generated by g^{(p-1)/d} mod p
    step = (p - 1) // d
    h = pow(g, step, p)
    subgroup = set(pow(h, i, p) for i in range(d))
    return subgroup

def coset_representatives(p, G, g):
    # Determine coset representatives for (Z/pZ)^*/G
    seen = set()
    reps = []
    for i in range(1, p):
        if i in seen:
            continue
        coset = {(i * x) % p for x in G}
        if not any(x in seen for x in coset):
            reps.append(i)
            seen |= coset
    return reps

def element_to_coset_idx(x, G, reps, p):
    if x == 0:
        return -1
    for idx, r in enumerate(reps):
        if any((r * g) % p == x for g in G):
            return idx
    raise ValueError(f"Element {x} not found in any coset")

def hyperaddition(p, G, reps, i, j):
    # Hyperaddition of cosets reps[i], reps[j]
    A = {(reps[i] * g) % p for g in G}
    B = {(reps[j] * g) % p for g in G}
    zero = 0
    result = set()
    for a in A:
        for b in B:
            s = (a + b) % p
            if s == zero:
                result.add(-1)
            else:
                result.add(element_to_coset_idx(s, G, reps, p))
    return result

def build_addition_table(p, G, reps):
    # Build addition table including zero element (-1)
    m = len(reps)
    # Use representatives in the table directly
    table = [[set() for _ in range(m + 1)] for _ in range(m + 1)]

    # Row/column headers based on representatives (including 0)
    headers = [0] + reps

    # Map coset index to representative
    idx_to_rep = {i: rep for i, rep in enumerate(reps)}
    idx_to_rep[-1] = 0

    for i in range(m + 1):
        for j in range(m + 1):
            if i == 0 and j == 0:
                table[i][j] = {0}
            elif i == 0:
                table[i][j] = {idx_to_rep[j-1]}
            elif j == 0:
                table[i][j] = {idx_to_rep[i-1]}
            else:
                result_indices = hyperaddition(p, G, reps, i-1, j-1)
                table[i][j] = {idx_to_rep[idx] for idx in result_indices}

    # Convert to pandas DataFrame for better visualization
    df_table = pd.DataFrame(table, index=[f"[{h}]" for h in headers], columns=[f"[{h}]" for h in headers])

    # Format the cells for readability
    def format_cell(cell_set):
        return "{" + ",".join([f"[{e}]" for e in sorted(cell_set)]) + "}"

    df_table = df_table.map(format_cell)

    return df_table

def format_cell_old(cell):
    return "{" + ",".join("0" if e == -1 else f"C{e}" for e in sorted(cell)) + "}"

def print_table_old(table):
    n = len(table) - 1
    headers = ["+"] + ["0"] + [f"C{i}" for i in range(n)]
    print('\t'.join(headers))
    for r in range(n + 1):
        row_label = "0" if r == 0 else f"C{r-1}"
        row_cells = [format_cell_old(table[r][c]) for c in range(n + 1)]
        print('\t'.join([row_label] + row_cells))

def hyper_sum_of_ones(p, G, reps, n_terms):
    # Compute 1⊞1⊞...⊞1 (n_terms)
    idx_1 = element_to_coset_idx(1, G, reps, p)
    # Need to build the table with original indices for calculation
    m = len(reps)
    table = [[set() for _ in range(m + 1)] for _ in range(m + 1)]
    for i in range(m + 1):
        table[0][i] = {i - 1} if i > 0 else {-1}
        table[i][0] = {i - 1} if i > 0 else {-1}
    for i in range(m):
        for j in range(m):
            table[i + 1][j + 1] = hyperaddition(p, G, reps, i, j)

    S = {idx_1}
    for _ in range(n_terms - 1):
        T = set()
        for a in S:
            row = a + 1 if a >= 0 else 0
            col = idx_1 + 1
            T |= table[row][col]
        S = T
    return S

def characteristic(p, G, reps, max_terms=2000):
    for n in range(1, max_terms + 1):
        if -1 in hyper_sum_of_ones(p, G, reps, n):
            return n
    return 0

def c_characteristic(p, G, reps, max_terms=2000):
    idx_1 = element_to_coset_idx(1, G, reps, p)
    for n in range(1, max_terms + 1):
        if idx_1 in hyper_sum_of_ones(p, G, reps, n + 1):
            return n
    return 0

# =====================
# Generalized with galois: GF(p^k) / G_d
# =====================

def validate_inputs(p, k, d):
    if not is_prime(p):
        raise ValueError("p must be prime")
    if k <= 0:
        raise ValueError("k must be a positive integer")
    if d <= 0:
        raise ValueError("d must be a positive integer")

def build_field_and_subgroup(p, k, d):
    if galois is None:
        raise ImportError("The 'galois' library is required. Install with 'pip install galois'.")
    validate_inputs(p, k, d)
    q = p ** k
    if (q - 1) % d != 0:
        raise ValueError(f"d must divide p^k - 1 = {q - 1}")
    GF = galois.GF(q)
    alpha = GF.primitive_element
    step = (q - 1) // d
    h = alpha ** step
    # Represent subgroup elements by their integer keys for hashing
    subgroup_keys = set(int(h ** i) for i in range(d))
    return GF, subgroup_keys

def gf_coset_representatives(GF, subgroup_keys):
    # Work with integer keys 1..q-1 to avoid unhashable GF elements
    q = GF.order
    seen_keys = set()
    reps_keys = []
    # Iterate starting from 1 to ensure the coset containing 1 is added first
    order = list(range(1, q))
    order.sort(key=lambda t: 0 if t == 1 else 1)
    for key in order:
        if key in seen_keys:
            continue
        # Coset as integer keys
        coset_keys = set(int(GF(key) * GF(gk)) for gk in subgroup_keys)
        if not any(ck in seen_keys for ck in coset_keys):
            reps_keys.append(key)
            seen_keys |= coset_keys
    # Return GF elements for display but we will still use keys internally elsewhere
    return [GF(k) for k in reps_keys]

def build_element_to_coset_index(GF, subgroup_keys, reps):
    # Map integer keys of field elements to coset index
    mapping = {}
    for idx, r in enumerate(reps):
        r_key = int(r)
        for gk in subgroup_keys:
            elem_key = int(GF(r_key) * GF(gk))
            mapping[elem_key] = idx
    return mapping

def gf_hyperaddition(GF, subgroup_keys, reps, element_to_idx, i, j):
    zero_key = 0
    r_i_key = int(reps[i])
    r_j_key = int(reps[j])
    A_keys = set(int(GF(r_i_key) * GF(gk)) for gk in subgroup_keys)
    B_keys = set(int(GF(r_j_key) * GF(gk)) for gk in subgroup_keys)
    result_indices = set()
    for a_key in A_keys:
        for b_key in B_keys:
            s_key = int(GF(a_key) + GF(b_key))
            if s_key == zero_key:
                result_indices.add(-1)
            else:
                result_indices.add(element_to_idx[s_key])
    return result_indices

def build_addition_table_gf(GF, subgroup_keys, reps):
    m = len(reps)
    element_to_idx = build_element_to_coset_index(GF, subgroup_keys, reps)
    # Use integer keys internally; 0 denotes zero element
    table = [[set() for _ in range(m + 1)] for _ in range(m + 1)]
    rep_keys = [int(r) for r in reps]
    # Headers are labels for readability
    def label_of_key(k):
        if k == 0:
            return "0"
        if k == 1:
            return "1"
        return str(GF(k))
    headers_labels = ["0"] + [label_of_key(k) for k in rep_keys]
    for i in range(m + 1):
        for j in range(m + 1):
            if i == 0 and j == 0:
                table[i][j] = {0}
            elif i == 0:
                table[i][j] = {rep_keys[j - 1]}
            elif j == 0:
                table[i][j] = {rep_keys[i - 1]}
            else:
                result_indices = gf_hyperaddition(GF, subgroup_keys, reps, element_to_idx, i - 1, j - 1)
                # Map coset indices to representative keys
                result_keys = set(rep_keys[idx] if idx >= 0 else 0 for idx in result_indices)
                table[i][j] = result_keys
    df_table = pd.DataFrame(table, index=[f"[{h}]" for h in headers_labels], columns=[f"[{h}]" for h in headers_labels])
    def format_cell(cell_set):
        return "{" + ",".join([f"[{label_of_key(k)}]" for k in sorted(cell_set)]) + "}"
    df_table = df_table.map(format_cell)
    return df_table

def gf_index_of_one(GF, subgroup_keys, reps):
    element_to_idx = build_element_to_coset_index(GF, subgroup_keys, reps)
    return element_to_idx[int(GF(1))]

def gf_hyper_sum_of_ones(GF, subgroup_keys, reps, n_terms):
    element_to_idx = build_element_to_coset_index(GF, subgroup_keys, reps)
    idx_1 = element_to_idx[int(GF(1))]
    S = {idx_1}
    for _ in range(n_terms - 1):
        T = set()
        for a in S:
            if a == -1:
                T.add(idx_1)
            else:
                T |= gf_hyperaddition(GF, subgroup_keys, reps, element_to_idx, a, idx_1)
        S = T
    return S

def gf_characteristic(GF, subgroup_keys, reps, max_terms=2000):
    for n in range(1, max_terms + 1):
        if -1 in gf_hyper_sum_of_ones(GF, subgroup_keys, reps, n):
            return n
    return 0

def gf_c_characteristic(GF, subgroup_keys, reps, max_terms=2000):
    idx_1 = gf_index_of_one(GF, subgroup_keys, reps)
    for n in range(1, max_terms + 1):
        if idx_1 in gf_hyper_sum_of_ones(GF, subgroup_keys, reps, n + 1):
            return n
    return 0

def main():
    while True:
        try:
            p = int(input("Enter a prime number p: "))
            k = int(input("Enter an integer k >= 1: "))
            validate_inputs(p, k, 1)
            q = p ** k
            print(f"p^k - 1 = {q - 1}")
            d = int(input(f"Enter divisor d of {q - 1}: "))
            if d <= 0 or (q - 1) % d != 0:
                print(f"d must divide {q - 1}. Try again.")
                continue
            break
        except ValueError as e:
            print(f"Invalid input: {e}")

    # Build GF(p^k) and subgroup of order d
    GF, subgroup_keys = build_field_and_subgroup(p, k, d)
    reps = gf_coset_representatives(GF, subgroup_keys)

    print(f"Field GF({p}^{k}) has order {p ** k}. Subgroup order d = {d}.")
    print(f"Coset representatives count: {len(reps)} (total quotient hyperfield order {len(reps) + 1})")

    table_df = build_addition_table_gf(GF, subgroup_keys, reps)
    print("Additive hyperoperation table (+):")
    display(table_df)

    # Optional: generalized characteristics via hyper-sums can be implemented similarly if needed
    char = gf_characteristic(GF, subgroup_keys, reps)
    cchar = gf_c_characteristic(GF, subgroup_keys, reps)
    print(f"Characteristic: {char}")
    print(f"C-characteristic: {cchar}")

if __name__ == "__main__":
    main()


Enter a prime number p: 5
Enter an integer k >= 1: 2
p^k - 1 = 24
Enter divisor d of 24: 3
Field GF(5^2) has order 25. Subgroup order d = 3.
Coset representatives count: 8 (total quotient hyperfield order 9)
Additive hyperoperation table (+):


,[0],[1],[2],[3],[4],[5],[7],[14],[15]
[0],{[0]},{[1]},{[2]},{[3]},{[4]},{[5]},{[7]},{[14]},{[15]}
[1],{[1]},"{[2],[4]}","{[3],[7],[15]}","{[4],[5],[14]}","{[0],[7],[15]}","{[2],[5],[15]}","{[1],[3],[14]}","{[2],[7],[14]}","{[1],[3],[5]}"
[2],{[2]},"{[3],[7],[15]}","{[3],[4]}","{[0],[5],[14]}","{[1],[5],[14]}","{[1],[2],[7]}","{[4],[5],[7]}","{[1],[2],[15]}","{[4],[14],[15]}"
[3],{[3]},"{[4],[5],[14]}","{[0],[5],[14]}","{[1],[2]}","{[2],[7],[15]}","{[3],[4],[7]}","{[1],[5],[7]}","{[3],[4],[15]}","{[1],[14],[15]}"
[4],{[4]},"{[0],[7],[15]}","{[1],[5],[14]}","{[2],[7],[15]}","{[1],[3]}","{[3],[5],[15]}","{[2],[4],[14]}","{[3],[7],[14]}","{[2],[4],[5]}"
[5],{[5]},"{[2],[5],[15]}","{[1],[2],[7]}","{[3],[4],[7]}","{[3],[5],[15]}","{[7],[14]}","{[1],[4],[15]}","{[0],[1],[4]}","{[2],[3],[14]}"
[7],{[7]},"{[1],[3],[14]}","{[4],[5],[7]}","{[1],[5],[7]}","{[2],[4],[14]}","{[1],[4],[15]}","{[14],[15]}","{[2],[3],[5]}","{[0],[2],[3]}"
[14],{[14]},"{[2],[7],[14]}","{[1],[2],[15]}","{[3],[4],[15]}","{[3],[7],[14]}","{[0],[1],[4]}","{[2],[3],[5]}","{[5],[15]}","{[1],[4],[7]}"
[15],{[15]},"{[1],[3],[5]}","{[4],[14],[15]}","{[1],[14],[15]}","{[2],[4],[5]}","{[2],[3],[14]}","{[0],[2],[3]}","{[1],[4],[7]}","{[5],[7]}"


Characteristic: 3
C-characteristic: 3
